In [12]:
import sys
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz as pgv
import random
from os import listdir
import pickle
import time
import instance
import statistics
from utils import read_param,add_lists,sub_lists, less_than, min_finish_time, find_index #Utility functions
import numpy as np
from deap import base,creator,tools,algorithms,gp
import operator
import math

In [2]:
train_set=[]

types=['j30','j60']
for typ in types:
  for i in range(1,49):
    train_set.append("./"+typ+'/'+typ+str(i)+"_1.sm")
    train_set.append("./"+typ+'/'+typ+str(i)+"_2.sm")


In [3]:

POP_SIZE=1024
NUM_GENERATIONS=25
INST_TYPE='j60'
MATING_PROB=0.5
MUTATION_PROB=0.3
SELECTION_POOL_SIZE=7
HOF_SIZE=1
HEIGHT_LIMIT = 6
MU=1024
LAMBDA=1024
GEN_MIN_HEIGHT=3
GEN_MAX_HEIGHT=5

In [4]:
def div(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1


pset = gp.PrimitiveSet("MAIN",10)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(div, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.renameArguments(ARG0="ES")
pset.renameArguments(ARG1="EF")
pset.renameArguments(ARG2="LS")
pset.renameArguments(ARG3="LF")
pset.renameArguments(ARG4="TPC")
pset.renameArguments(ARG5="TSC")
pset.renameArguments(ARG6="RR")
pset.renameArguments(ARG7="AvgRReq")
pset.renameArguments(ARG8="MaxRReq")
pset.renameArguments(ARG9="MinRReq")
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
counts=0
def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    
    func = toolbox.compile(expr=individual)
    sumv=0
    for i in range(len(train_set)):
        
        file=train_set[i]
        
        inst=instance.instance(file,use_precomputed=True)
        priorities=[0]*(inst.n_jobs+1)
        for j in range(1,inst.n_jobs+1):
            priorities[j]=func(inst.earliest_start_times[j],inst.earliest_finish_times[j],inst.latest_start_times[j],inst.latest_finish_times[j],inst.mtp[j],inst.mts[j],inst.rr[j],inst.avg_rreq[j],inst.max_rreq[j],inst.min_rreq[j])

        frac,makespan=inst.parallel_sgs(option='forward',priority_rule='',priorities=priorities)
        sumv+=frac
    
    return (sumv/len(train_set),)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=GEN_MIN_HEIGHT, max_=GEN_MAX_HEIGHT)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=SELECTION_POOL_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=GEN_MIN_HEIGHT, max_=GEN_MAX_HEIGHT)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=HEIGHT_LIMIT))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=HEIGHT_LIMIT))

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)
pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(HOF_SIZE)
pop, log = algorithms.eaMuPlusLambda(pop, toolbox,MU,LAMBDA, MATING_PROB, MUTATION_PROB, NUM_GENERATIONS, stats=mstats,halloffame=hof, verbose=True)


   	      	                                 fitness                                 	                      size                     
   	      	-------------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max     	min     	nevals	std      	avg    	gen	max	min	nevals	std    
0  	1024  	0.245142	0  	0.306941	0.173547	1024  	0.0301487	22.3145	0  	63 	4  	1024  	12.8076
1  	821   	0.20355 	1  	0.253415	0.173547	821   	0.0157159	21.7676	1  	61 	2  	821   	12.1611
2  	834   	0.184907	2  	0.208535	0.173051	834   	0.00676349	20.6328	2  	61 	2  	834   	12.3812
3  	827   	0.177169	3  	0.189417	0.172503	827   	0.00348525	17.2129	3  	61 	2  	827   	7.89071
4  	838   	0.173676	4  	0.180078	0.17207 	838   	0.000920015	17.7207	4  	41 	6  	838   	6.19287
5  	797   	0.173089	5  	0.173996	0.17207 	797   	0.000419875	22.8418	5  	38 	6  	797   	6.26517
6  	815   	0.172533	6  	0.173547	0.17207 	815   	0.000318498	24.9668	6

In [5]:
file=open('./evolved_funcs/best_funcs3','wb')
hof=pickle.dump(hof,file)
file.close()


In [13]:
for hof_index in range(HOF_SIZE):
    nodes, edges, labels = gp.graph(hof[hof_index])
    print("Function ", hof[hof_index])
    test_type=['j30','j60','j90','j120']
    sum_total_dev=0
    sum_counts=0
    for typ in test_type:
        total_dev_percent,makespan,total_dev,count=statistics.evaluate_custom_rule(instance.instance,toolbox.compile(expr=hof[hof_index]),inst_type=typ,mode='parallel',option='forward')
        print(typ,total_dev_percent,makespan)
        log_file=open('results_log.txt','a+')
        log_file.write(str(hof[hof_index])+" : \n              "+INST_TYPE+"         "+typ+"         "+str(len(train_set))+"               "+str(NUM_GENERATIONS)+"          "+str(MATING_PROB)+"           "+str(MUTATION_PROB)+"         "+str(round(total_dev_percent,2))+"        "+str(makespan)+"       \n\n")
        log_file.close()
        sum_total_dev+=total_dev
        sum_counts+=count
    print("Aggregate % ",(sum_total_dev*100)/sum_counts)


    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw("./gp_trees/"+str(round(total_dev_percent,2))+"__3.png")


Function  add(add(LS, LF), min(div(add(LS, LF), sub(div(neg(LF), mul(MaxRReq, TSC)), min(mul(MaxRReq, EF), mul(LS, LS)))), neg(AvgRReq)))
./j30/j309_3.sm (0.5625, 75) 18.329691274224558                                                      17.960909597368662                                                                            (0.0, 51) 18.730802475244825                    (0.42857142857142855, 90) 18.87455900754932                    (0.4745762711864407, 87) 18.645798822900634                   (0.11363636363636363, 49) 18.099467013749095                   (0.19047619047619047, 50) 18.242424438484242                    (0.0, 52) 17.92038213186094                   18.381022521557014                    (0.015625, 65) 18.47931989963749                   18.59457694502528                    (0.1346153846153846, 59) 18.088527879264923                                                          (0.15555555555555556, 52) 18.127930953293095                   
384 j30 files read
j30 18.3296

KeyboardInterrupt: 